# 문제 배경
한 과일 선별장은 여러 농장에서 수확된 혼합 과일을 받아 등급 A 품질의 과일만 골라 판매한다. 선별 후 등급 A의 수율은 농장마다 다르며, 현재는 고정값으로 가정하고 있다. 그러나 실제 데이터를 보면 일자별로 수율이 유동적이며, 농장별 품질에도 편차가 존재한다.

따라서, 각 농장이 공급한 과일에서 얼마나 높은 비율의 A급 과일이 나오는지를 데이터 기반으로 추정/최적화하여, 수익 예측 정확도와 공급 전략을 개선하고자 한다.

## 최적화 과제
30일간의 실제 수확량, 구매단가, 판매단가, 수익 데이터를 바탕으로
각 농장별 등급 A 비율(수율)을 추정하라.

⚠️ 단, “등급A_비율(%)”은 현재는 임의 수치로 되어 있으므로,
이를 조정해 실제 수익과 최대한 부합하도록 하는 최적화 대상이 된다.

In [1]:
import pandas as pd
import seaborn as sns

df = pd.read_csv('./data/농산물_선별수율_최적화문제.csv', encoding = 'utf-8')
df.head(10)

,일자,농장,수확량(톤),구매단가(만원/톤),등급A_비율(%),판매단가(만원/톤)
0,2025-04-01,농장A,6,1.5,25,5
1,2025-04-01,농장B,28,1.2,18,5
2,2025-04-01,농장C,14,1.8,30,5
3,2025-04-01,농장D,7,0.9,12,5
4,2025-04-01,농장E,20,1.3,20,5
5,2025-04-02,농장A,6,1.5,25,5
6,2025-04-02,농장B,18,1.2,18,5
7,2025-04-02,농장C,10,1.8,30,5
8,2025-04-02,농장D,10,0.9,12,5
9,2025-04-02,농장E,23,1.3,20,5


# 최적화 코드

In [13]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

df = pd.read_csv('./data/농산물_선별수율_최적화문제.csv', encoding = 'utf-8')

# 농장목록
# farms = list(df['농장'].unique())
farms = df['농장'].unique()

# 초기 수율
initial_ratio = [df[df['농장'] == farm]['등급A_비율(%)'].iloc[0] for farm in farms]
initial_ratio

[25, 18, 30, 12, 20]

In [ ]:
# 최적화 대상 함수: 일자별 수익 총합의 표준편차(분산)을 최소화
# 예측 수익 = 수확량*수율*판매단가 - 수확량*구매단가

df_temp = df.copy()

for i, farm in enumerate(farms):
    df_temp.loc[df_temp['농장'] == farm, "예측수익"] = df_temp['수확량(톤)'] * (initial_ratio[i]/100 * df_temp["판매단가(만원/톤)"] - df_temp["구매단가(만원/톤)"]
    